# Data Augmentation
- Demonstrates how to apply data augmentation (a technique to increase the diversity of training set by applying transformations, such as image rotation)
- Full tutorial reference: https://www.tensorflow.org/tutorials/images/data_augmentation

### Import TensorFlow and necessary libraries

In [3]:
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.9.1


### Download the dataset